## GPU確認  

下記セルを実行し、出力結果に”device_type:GPU”が存在するか確認する。

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4830908632639579193,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5186304096
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16380112720165943509
 physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"]

## 学習・推論確認

##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 初心者のための TensorFlow 2.0 入門

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs-l10n](https://github.com/tensorflow/docs-l10n)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

この短いイントロダクションでは [Keras](https://www.tensorflow.org/guide/keras/overview) を使って下記のことを行います。

1. 画像を分類するニューラルネットワークを構築する
2. このニューラルネットワークを訓練する
3. そして最後に、モデルの正解率を評価する

このファイルは [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) の notebook ファイルです。 Python プログラムはブラウザ上で直接実行されます。TensorFlow を学んだり使ったりするには最良の方法です。Google Colab のnotebook の実行方法は以下のとおりです。

1. Pythonランタイムへの接続：メニューバーの右上で「接続」を選択します。
2. ノートブックのコードセルをすべて実行：「ランタイム」メニューから「すべてのセルを実行」を選択します。

TensorFlow 2 のパッケージをダウンロードしてインストールします。プログラムに TensorFlow をインポートします。

Note: TensorFlow 2 パッケージをインストールするためには `pip` をアップグレードしてください。詳細は [インストール](https://www.tensorflow.org/install)を参照ください。

In [2]:
import tensorflow as tf

[MNIST データセット](http://yann.lecun.com/exdb/mnist/)をロードして準備します。サンプルを整数から浮動小数点数に変換します。

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

層を積み重ねて`tf.keras.Sequential`モデルを構築します。訓練のためにオプティマイザと損失関数を選びます。

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

モデルはそれぞれの標本について、クラスごとに"[ロジット](https://developers.google.com/machine-learning/glossary#logits)"や"[対数オッズ比](https://developers.google.com/machine-learning/glossary#log-odds)"と呼ばれるスコアを算出します。

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.38498694, -0.39526224,  0.22535872, -0.15563166,  0.04310162,
        -0.3473403 , -0.28732288, -0.19363233,  0.30940607, -0.3540815 ]],
      dtype=float32)

`tf.nn.softmax` 関数はクラスごとにこれらのロジットを "確率" に変換します。

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.15243155, 0.06985813, 0.1299419 , 0.08877429, 0.10829189,
        0.07328738, 0.07782057, 0.0854641 , 0.14133525, 0.072795  ]],
      dtype=float32)

Note: `tf.nn.softmax` をネットワークの最終レイヤーに対する活性化関数として組み込むことは可能です。これはモデルの出力に対して直接的に解釈できるようにする一方で、ソフトマックスの出力を用いる場合、すべてのモデルに対して正確で数値的安定に損失を計算することが不可能なため、このアプローチは推奨されません。

`losses.SparseCategoricalCrossentropy` 損失関数は、ロジットと `True` の インデックスに関するベクトルを入力にとり、それぞれの標本についてクラスごとに損失のスカラーを返します。

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

この損失は、クラスが正しい確率の対数をとって符号を反転させたものです。この値はモデルがこのクラスが正しいと確信しているときに 0 になります。

この訓練されていないモデルはランダムに近い確率 (それぞれのクラスについて 1/10) を出力します、最初の損失は `-tf.log(1/10) ~= 2.3` に近い値になるはずです。

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.6133668

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

`Model.fit` メソッドは損失を最小化するようにモデルのパラメータを調整します。

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


   1/1875 [..............................] - ETA: 0s - loss: 2.3686 - accuracy: 0.0938

  36/1875 [..............................] - ETA: 2s - loss: 1.5747 - accuracy: 0.5530

  72/1875 [>.............................] - ETA: 2s - loss: 1.1614 - accuracy: 0.6793

 106/1875 [>.............................] - ETA: 2s - loss: 0.9773 - accuracy: 0.7305

...

1740/1875 [==========================>...] - ETA: 0s - loss: 0.0751 - accuracy: 0.9767

1777/1875 [===========================>..] - ETA: 0s - loss: 0.0749 - accuracy: 0.9767

1813/1875 [============================>.] - ETA: 0s - loss: 0.0748 - accuracy: 0.9766

1847/1875 [============================>.] - ETA: 0s - loss: 0.0747 - accuracy: 0.9767

1875/1875 [==============================] - 3s 1ms/step - loss: 0.0744 - accuracy: 0.9768


`Model.evaluate` メソッドはモデルの性能を検査します。これには通常 "[検証用データセット](https://developers.google.com/machine-learning/glossary#validation-set)" または "[テストデータセット](https://developers.google.com/machine-learning/glossary#test-set)" を用います。

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0772 - accuracy: 0.9772


[0.07715432345867157, 0.9771999716758728]

この画像識別器はこのデータセットにおいておよそ 98% の正解率にまで訓練されました。更に学ぶためには [TensorFlow tutorials](https://www.tensorflow.org/tutorials/) を参照ください。

モデルが確率を返すようにしたい場合には、モデルをラップしてソフトマックス関数を適用してください。

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[9.44173895e-09, 6.94358404e-10, 3.79947642e-06, 4.39701034e-05,
        1.80126985e-12, 1.67197212e-09, 2.21749859e-15, 9.99951243e-01,
        5.67929170e-08, 9.21883554e-07],
       [7.79088623e-07, 3.74176976e-04, 9.99573529e-01, 4.89070990e-05,
        1.02540784e-14, 6.65212610e-07, 3.46177728e-07, 2.32120841e-14,
        1.70997953e-06, 2.19384921e-13],
       [2.41820476e-06, 9.98024464e-01, 1.30830987e-04, 3.87457731e-05,
        1.14681439e-04, 1.48148529e-05, 1.03454922e-05, 1.57439278e-03,
        8.68293500e-05, 2.53524536e-06],
       [9.99934912e-01, 7.94180867e-12, 3.35007462e-05, 3.67401071e-07,
        9.42708525e-08, 2.27556725e-07, 1.35170385e-05, 3.18891853e-06,
        2.56671262e-09, 1.42238232e-05],
       [1.92597008e-06, 6.02066841e-10, 4.11342489e-06, 3.30409513e-08,
        9.90220666e-01, 6.79421674e-08, 3.00520554e-07, 8.96678102e-05,
        6.58433947e-08, 9.68321506e-03]], dtype=float32)>